In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader


class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file)
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        sample = self.df.iloc[idx]
        
        features = torch.tensor(sample[1:].values, dtype=torch.float32)
        label = torch.tensor(sample[0], dtype=torch.float32)
        
        return {"features": features, "label": label}
            
        

In [5]:
my_dataset = CustomDataset(csv_file='../../data/traffic_accidents/traffic_accidents.csv')
# for i, sample in enumerate(my_dataset):
#     print(i, sample["features"], sample["label"])

dataloader = DataLoader(my_dataset, batch_size=50, shuffle=True)


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Один выходной нейрон для регрессии
    
    def forward(self, x):
        return self.linear(x)
    

model = LinearRegressionModel(input_dim=len(my_dataset.df.columns) - 1)  # Количество фичей в данных
# nn.utils.clip_grad_norm_(model.parameters(), max_norm=10.0)

# Используем Mean Squared Error для регрессии
criterion = nn.MSELoss()

# Используем оптимизатор SGD
# optimizer = optim.SGD(model.parameters(), lr=0.0001)
optimizer = optim.Adam(model.parameters(), lr=0.01)


# 3. Процесс обучения
epochs = 100
for epoch in range(epochs):
    epoch_loss = 0
    for i, batch in enumerate(dataloader):
        features = batch["features"]
        labels = batch["label"]

        # Прямой проход
        outputs = model(features)
        # print(f"Epoch #{epoch}, Batch #{i}", outputs, sep='\n')

        # Вычисление потерь
        loss = criterion(outputs.squeeze(), labels)  # .squeeze() для удаления лишних измерений

        # Обратный проход и обновление параметров
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    # Выводим прогресс
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss / len(dataloader):.4f}")

Epoch [1/100], Loss: 61.1101
Epoch [2/100], Loss: 25.1168
Epoch [3/100], Loss: 18.2598
Epoch [4/100], Loss: 14.5633
Epoch [5/100], Loss: 12.1887
Epoch [6/100], Loss: 10.4661
Epoch [7/100], Loss: 9.3202
Epoch [8/100], Loss: 8.4737
Epoch [9/100], Loss: 7.7393
Epoch [10/100], Loss: 7.1015


KeyboardInterrupt: 